# Creating and Testing Tools for the FPL Gaffer

## FPL API Data Retrieval Tool

In [1]:
import httpx

In [2]:
base_url = "https://fantasy.premierleague.com/api"
client = httpx.Client()

### Retrieving Bootstrap Data

In [3]:
# Retrieve bootstrap data from FPL API endpoint
bootstrap_endpoint = f"{base_url}/bootstrap-static/"
response = client.get(bootstrap_endpoint)

# Ensure the request was successful
response.raise_for_status()
response_data = response.json()

# Display the keys of the response data
response_data.keys()

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'element_stats', 'element_types', 'elements'])

### Getting Player Data

In [4]:
player_data = response_data["elements"]

# Display the first 10 players
for player in player_data[:10]:
    print(f"ID: {player["id"]}, Name: {player["web_name"]}, Team: {player["team"]}, Position: {player["element_type"]}")

ID: 1, Name: Raya, Team: 1, Position: 1
ID: 2, Name: Arrizabalaga, Team: 1, Position: 1
ID: 3, Name: Hein, Team: 1, Position: 1
ID: 4, Name: Setford, Team: 1, Position: 1
ID: 5, Name: Gabriel, Team: 1, Position: 2
ID: 6, Name: Saliba, Team: 1, Position: 2
ID: 7, Name: Calafiori, Team: 1, Position: 2
ID: 8, Name: J.Timber, Team: 1, Position: 2
ID: 9, Name: Kiwior, Team: 1, Position: 2
ID: 10, Name: Lewis-Skelly, Team: 1, Position: 2


### Getting Gameweek Data

In [7]:
gw_data = response_data.get("events", [])

# Since no gameweek is active now as the season hasn't started,
# let's get the other data than current gameweek

gw_dict = {}
for gw in gw_data:
    # Get the details for the firt gameweek
    gw_dict["gamweek"] = gw.get("id") if gw else None
    gw_dict["deadline"] = gw.get("deadline_time") if gw else None
    gw_dict["finished"] = gw.get("finished") if gw else None
    break

gw_dict

{'gamweek': 1, 'deadline': '2025-08-15T17:30:00Z', 'finished': False}

### Getting Fixture Data

In [11]:
fixture_endpoint = f"{base_url}/fixtures/"

# Get fixture data
fixtures_data = client.get(fixture_endpoint).json()

# Display the first entry from the fixtures data
fixtures_data[0]

{'code': 2561895,
 'event': 1,
 'finished': False,
 'finished_provisional': False,
 'id': 1,
 'kickoff_time': '2025-08-15T19:00:00Z',
 'minutes': 0,
 'provisional_start_time': False,
 'started': False,
 'team_a': 4,
 'team_a_score': None,
 'team_h': 12,
 'team_h_score': None,
 'stats': [],
 'team_h_difficulty': 3,
 'team_a_difficulty': 5,
 'pulse_id': 124791}

## FPL News Searcher Tool

In [27]:
from dotenv import load_dotenv, find_dotenv
import os
from tavily import TavilyClient
from langchain.document_loaders import UnstructuredURLLoader

In [29]:
# Load environment variables
_ = load_dotenv()

tavily_api_key = os.getenv("TAVILY_API_KEY")

# Creating search client
tavily_client = TavilyClient(
    api_key=tavily_api_key
)

### Testing search workflow with sample queries

In [ ]:
documents = []
queries = [
    "Fantasy Premier League player injuries and suspensions",
    "FPL Scout best tips and advice latest",
    "FPL differential picks low ownership players latest"
]

for query in queries:
    # Get the tavily api search results for each query
    response = tavily_client.search(
        query=query,
        max_results=3,
        topic="news",
        include_raw_content=False
    )

    # Get the urls from the search results
    urls = [item["url"] for item in response["results"]]

    # Fetch and parse content from the URLs
    loader = UnstructuredURLLoader(urls=urls)
    docs = loader.load()

    # Add metadata from tavily to each document
    for i, doc in enumerate(docs):
        doc.metadata.update({
            "source": urls[i],
            "query": query
        })

    # Added the docs to the document list
    documents.extend(docs)


In [33]:
print(documents[0].page_content)

NINE things we learned from weekend's pre-season matches

Ben Bloom 04 Aug 2025

Ben Bloom reports on the key takeaways following the latest round of fixtures

Football writer Ben Bloom picks out the key takeaways from the previous weekend's matches.

Manchester United were crowned the 2025 Premier League Summer Series champions, while Son Heung-min’s seemed to have played his final match as a Tottenham Hotspur player.

Here are nine key takeaways from last weekend.

Mbeumo and Cunha united without a No 9

Much of the anticipation ahead of the new season at Old Trafford stems from the big-money signings of Bryan Mbeumo and Matheus Cunha.

For the first time, in a 2-2 Premier League Summer Series draw against Everton the pair started a match together and impressed, despite neither putting their name on the scoresheet.

While Mbeumo lined up on the right, it was interesting to see Cunha given an opportunity to play as something of a false 9. In truth, the front three, which also featured